In [47]:
# Import the required libraries
import hvplot.pandas
import pandas as pd
import warnings
from config import geoapify_key
import requests
import hvplot.pandas

In [29]:
# Turn off warning messages
warnings.filterwarnings('ignore')

In [30]:
# Import csv to dataframe
city_data_df = pd.read_csv('weather_data/cities.csv')
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Jamestown,US,2022-12-09 20:42:23,42.0970,-79.2353,35.24,43,40,11.50
1,1,Cape Town,ZA,2022-12-09 20:42:24,-33.9258,18.4232,68.41,84,20,8.05
2,2,Ous,RU,2022-12-09 20:42:25,60.9058,61.5186,7.11,98,99,2.55
3,3,Birao,CF,2022-12-09 20:42:25,10.2849,22.7882,71.80,23,5,6.46
4,4,Ilulissat,GL,2022-12-09 20:42:25,69.2167,-51.1000,39.22,45,75,14.97


In [31]:
# GeoViews requires data to be in integer or fp decimal, check data types
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [32]:
# Configure map plot 1
map_plot_1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM'
)
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]

In [33]:
# Configure map plot 2
map_plot_2 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 700,
    frame_height = 500
)
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]

In [34]:
# Configure map plot 3
map_plot_3 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 700,
    frame_height = 500,
    size = 'Max Temp',
    scale = 0.5,
    color = 'City'
)
map_plot_3

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp)

In [35]:
# Configure map plot 4
map_plot_4 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'EsriNatGeo',
    frame_width = 700,
    frame_height = 500,
    size = 'Max Temp',
    scale = 0.5,
    color = 'City'
)
map_plot_4

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp)

In [36]:
# Configure map plot 5
map_plot_5 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'CartoDark',
    frame_width = 700,
    frame_height = 500,
    size = 'Wind Speed',
    color = 'City'
)
map_plot_5

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Wind Speed)

In [37]:
# Ask customer to add a min and max temp
min_temp = float(input('What is the minimum temperature you would like for your trip?'))
max_temp = float(input('What is the maximum temperature you would like for your trip?'))

In [38]:
preferred_cities_df = city_data_df.loc[
    (city_data_df['Max Temp'] <= max_temp) & (city_data_df['Max Temp'] >= min_temp)
].dropna()
preferred_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Cape Town,ZA,2022-12-09 20:42:24,-33.9258,18.4232,68.41,84,20,8.05
3,3,Birao,CF,2022-12-09 20:42:25,10.2849,22.7882,71.80,23,5,6.46
5,5,Hermanus,ZA,2022-12-09 20:39:53,-34.4187,19.2345,66.79,85,0,5.55
6,6,Port Alfred,ZA,2022-12-09 20:42:26,-33.5906,26.8910,68.18,94,97,1.99
11,11,Tatawin,TN,2022-12-09 20:42:28,32.9297,10.4518,69.21,30,99,9.42


In [39]:
preferred_cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 193 entries, 1 to 590
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     193 non-null    int64  
 1   City        193 non-null    object 
 2   Country     193 non-null    object 
 3   Date        193 non-null    object 
 4   Lat         193 non-null    float64
 5   Lng         193 non-null    float64
 6   Max Temp    193 non-null    float64
 7   Humidity    193 non-null    int64  
 8   Cloudiness  193 non-null    int64  
 9   Wind Speed  193 non-null    float64
dtypes: float64(4), int64(3), object(3)
memory usage: 16.6+ KB


In [40]:
# Create copy of filtered df and add column to store hotel info in
hotel_df = preferred_cities_df[['City', 'Country', 'Max Temp', 'Lat', 'Lng']].copy()
hotel_df['Hotel Name'] = ''
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Cape Town,ZA,68.41,-33.9258,18.4232,
3,Birao,CF,71.80,10.2849,22.7882,
5,Hermanus,ZA,66.79,-34.4187,19.2345,
6,Port Alfred,ZA,68.18,-33.5906,26.8910,
11,Tatawin,TN,69.21,32.9297,10.4518,


In [41]:
# Set initial parameters to search for a hotel
radius = 5000
params = {
    'categories':'accommodation.hotel',
    'apiKey':geoapify_key
}

In [45]:
# Iterate through the hotel_df rows
for index, row in hotel_df.iterrows():
    #Get lat and lon
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add current city's lats and lngs to params
    params['filter'] = f'circle:{longitude},{latitude},{radius}'

    base_url = "https://api.geoapify.com/v2/places"
    name_address = requests.get(base_url, params=params).json()

    # Get first hotel and store name
    try:
        hotel_df.loc[index, 'Hotel Name'] = name_address['features'][0]['properties']['name']
    except (KeyError, IndexError):
        # If no hotel is found, set as No Hotel Found
        hotel_df.loc[index, 'Hotel Name'] = 'No hotel found'


In [46]:
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Cape Town,ZA,68.41,-33.9258,18.4232,DoubleTree by Hilton Hotel Cape Town - Upper E...
3,Birao,CF,71.80,10.2849,22.7882,No hotel found
5,Hermanus,ZA,66.79,-34.4187,19.2345,Abalone Guest Lodge
6,Port Alfred,ZA,68.18,-33.5906,26.8910,Ferryman
11,Tatawin,TN,69.21,32.9297,10.4518,نزل اميلكار


In [53]:
# Configure map
hotel_map = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 700,
    frame_height = 500,
    size = 'Max Temp',
    scale = 1,
    color = 'City',
    hover_cols = ['Hotel Name', 'Country']
)
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country)